In [88]:
import nbimporter
import requests
import os
import json
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
from common import Website
from IPython.display import Markdown, display, update_display

In [83]:
load_dotenv()

openapi_key = os.getenv("OPENAI_API_KEY")

MODEL="gpt-4o-mini"

openai = OpenAI()

In [68]:
system_prompt = "You are an assistant that can choose relevant links for a brochure of a company from the given list of links of a website.\n"
system_prompt += "Give response in json format as below: \n" 
system_prompt +=              """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

print (system_prompt)

You are an assistant that can choose relevant links for a brochure of a company from the given list of links of a website.
Give response in json format as below: 

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [69]:
def get_user_prompt():
    user_prompt = "Here are the links of a website"
    user_prompt += "\n".join(web.links)
    user_prompt += "Please decide which links are relevant to create brocure for the company. Skip pages like Terms of Service, Privacy. Respond with the full https URL in JSON format"
    return user_prompt

In [75]:
def get_links(url):

    web = Website(url)
    user_prompt = get_user_prompt()
    openai = OpenAI()
    messages = [{"role" : "system", "content": system_prompt},{"role" : "user" , "content": user_prompt}]
    response = openai.chat.completions.create(model=MODEL, messages = messages)
    links = response.choices[0].message.content
    return json.loads(links)

{'links': [{'type': 'about page', 'url': 'https://www.explodingkittens.com/pages/about'}, {'type': 'contact page', 'url': 'https://www.explodingkittens.com/pages/contact'}, {'type': 'blog', 'url': 'https://www.explodingkittens.com/blog'}, {'type': 'products page', 'url': 'https://www.explodingkittens.com/products/throw-throw-burrito-original-edition'}, {'type': 'animations', 'url': 'https://www.explodingkittens.com/animations'}, {'type': 'comics', 'url': 'https://www.explodingkittens.com/comics'}, {'type': 'games', 'url': 'https://www.explodingkittens.com/games'}, {'type': 'books', 'url': 'https://www.explodingkittens.com/books'}]}


In [108]:
#get contents of relevant pages of website
def get_website_summary(url):
    summary = Website(url).get_contents()
    links = get_links(url)
    
    for link in links["links"]:
        link_contents = Website(link["url"]).get_contents()
        summary += link_contents
    return summary


In [109]:
brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short, humorous and quirky brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(url):
    brochure_user_prompt = f"You are looking at the website {url} \n. Please create a short brochure for the company. \nHere is the content of several relevant pages of a website \n"
    brochure_user_prompt += get_website_summary(url)
    return brochure_user_prompt
    

In [122]:
# make call to gpt to create brochhure
def create_brochure(url):
    messages = [{"role": "system", "content": brochure_system_prompt}, {"role": "user", "content": brochure_user_prompt}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    # if stream=True is passed
    # for chunk in response:
    #     display(Markdown(chunk.choices[0].delta.content))
    
    result = response.choices[0].message.content
    return result



In [123]:
url = "https://theoatmeal.com/"
display(Markdown(create_brochure(url)))

# 📜 Welcome to The Oatmeal: Where Creativity Meets Cats! 🐱

---

## **What’s Cooking at The Oatmeal?**

At The Oatmeal, we transform unremarkable moments into delightful bursts of laughter! Founded by the witty and wonderfully eccentric Matthew Inman, we serve you comics, quizzes, and love for all things nerdy. It’s like your grandma’s kitchen but without the awkward Thanksgiving conversations (unless you want them).

### **Who Are Our Customers? (Hint: Everyone!)**

1. **Cat People:** You have a special place in our hearts (and comics).
2. **Coffee Addicts:** For those that fuel their genius with caffeine, we get you!
3. **Nerds & Geeks:** Whether you're into cosplay or coding, you're our *kinda* people.
4. **Lovers of Food:** Because who doesn't love a cartoon about everyday culinary disasters?

---

## **Company Culture: Quirky, Creative, and Cat-centric**

Here at The Oatmeal, we believe in embracing your weirdness! Our office looks like a rainbow exploded, and somewhere, there’s definitely at least one cat lounging on a pile of designer pillows.

### **Core Values:**
- **Celebrate the Absurd:** Life is way too short for boring. We thrive on creativity and hilarious ideas!
- **Punny Business:** If it doesn't make you giggle, why bother?
- **Cats, Cats, and More Cats:** It’s a scientific fact that creativity increases with feline companionship. Okay, we made that up, but it feels true.

---

## **Join Our Realm of Brilliance! 🚀 Careers at The Oatmeal**

Looking to join a team where your wittiness is not just appreciated but celebrated? Here’s what you can expect when you hop on-board:

### **Job Perks:**
- **Flexible Hours:** Work when your brain is the most caffeinated!
- **Creative Freedom:** Write your comedy gold or draw the next meme masterpiece. No judgment, just applause!
- **Monthly “Nerd Out” Days:** Game on or indulge in binge-watching your favorite shows with our crew!

### **Available Positions:**
- Professional Cat Cuddler (with a side of comic drawing, obviously)
- Pun Developer (must know at least 100 puns or possess the will to learn)
- Head of Quirk (responsible for maintaining the absurdity level at work)

---

## 🍵 Why Choose The Oatmeal?

Because it’s not just oatmeal; it’s a delicious bowl of laughter! We’re not saying we’ll change your life, but you will definitely chuckle or snort (particularly if you're a coffee drinker) at least once a day if you hang around us.

---

**Ready to join the madness?** Head over to our careers page! Embrace your weirdness with open arms, and let’s create something hilarious together! 

---

🛒 **Shop Our Wares:** Swing by our merch section for comics, books, and cat-illustrated items that say, “Yes, I can be quirky in public!”

**The Oatmeal:** Where every day is a pun-derful adventure! 

---

In [120]:
translater_system_prompt = "You are a translater who can translate from English to Spanish"

def get_translater_user_prompt(content): 
    user_prompt = "Given the brochure of a company in English, translate it in Spanish language\n"
    user_prompt += content
    return user_prompt
    

def translate_in_spanish(content):
    messages = [{"role": "system", "content": translater_system_prompt}, {"role": "user", "content": get_translater_user_prompt(content)}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content



In [127]:
url = "https://theoatmeal.com/"
display(Markdown(translate_in_spanish(create_brochure(url))))

# ¡Bienvenido a The Oatmeal: Un Sitcom Serio para Personas más Inteligentes!

**¿Quiénes Somos?**  
Un grupo genial de escritores y dibujantes que creen que un grano de humor puede hacer una gran diferencia. Fundado por el aparentemente inmortal Matthew Inman, The Oatmeal está aquí para hacerte reír mientras te proporciona una variada gama de contenido educativo que va desde **ilustraciones y cómics peculiares** hasta **bombas de conocimiento** que explotan con sabiduría inesperada.

### ¡Nuestros Clientes son No Monotónicos!

Nuestros fans son esas personas fabulosas que, al igual que nosotros, creen en el poder de la risa mezclada con el aprendizaje. Ya seas un analista de datos amante del café, un padre o madre que se queda en casa buscando risas, o un entusiasta de los gatos inmerso en memes de gatos, ¡tenemos algo solo para ti! Nuestra audiencia diversa incluye:

- **Nerds y Geeks**: Desde matemáticos hasta magos del código.
- **Amantes de los Gatos**: Porque, ¿quién no disfruta de un compañero ronroneador mientras se desplaza por memes?
- **Snobs del Café**: ¡Tus quejas alimentadas por cafeína han encontrado su pareja en nuestro humor!

### Carreras en The Oatmeal: Donde lo Ingenioso se Encuentra con lo Resiliente

¿Buscas unirte a una comunidad que celebra la rareza como la forma más alta de inteligencia? Consulta nuestras oportunidades laborales donde podrás trabajar junto a algunas de las mentes más peculiares.

- **Creadores de Cómics**: ¿Tienes talento para el humor? ¡Podrías convertir lo mundano en momentos dignos de meme!
- **Magos de las Palabras**: Escritores prolíficos que pueden convertir un simple “hola” en una epifanía.
- **Cultivadores de Gatos (y otras mascotas)**: Está bien, puede que solo necesites llevar a tu mascota a la oficina; ¡se recomienda encarecidamente para la moral!

### Nuestra Cultura Empresarial: ¡Donde la Diversión se Encuentra con la Funcionalidad!

En The Oatmeal, creemos en una mezcla de creatividad, colaboración y cafeína. Imagina un espacio de trabajo donde:

- **Sesiones de Lluvia de Ideas**: Estas a menudo se convierten en competencias de baile espontáneas. ¡Sí, nos tomamos nuestra diversión *muy* en serio!
- **Horarios de Trabajo Flexibles**: Llegar a la oficina a la hora que dicte tu espíritu animal interno, ¡siempre y cuando cumplas con tus plazos!
- **Salud y Bienestar**: Sesiones semanales de yoga de la risa (en serio) para mantener altos los ánimos y trabajar los abdominales.

### ¿Por Qué Elegir The Oatmeal?

Si estás buscando un lugar que celebre la inteligencia envuelta en humor, ¡no busques más! Aquí, producimos contenido que te hace sonreír, pensar y, a veces, llorar (de la risa). ¡Estamos contigo cuando se trata de compartir ideas que hacen que el mundo sea un poco más brillante y mucho más divertido!

---

**¡Únete a nosotros en The Oatmeal—donde el humor y la sabiduría chocan en una espectacular exhibición de hilaridad, aprendizaje y apreciación felina!** 🐱✨ 

Recuerda: La vida es demasiado corta para contenido aburrido. ¡Dale un toque especial con un poco de avena!